---
---

**PSTB Bootcamp - Hackathon 1 - Subject 3**: *Exploring Employee Attrition and Performance in a Corporate Environment (IBM HR Analytics)*


---


**Date** : 23/10/2025


---


**Team**: YC Dynamics (Yacine Fares, Clément Philbert)

---
---

# 1 ) Data Loading and Preprocessing

## a. Downloading the IBM HR Analytics dataset and extracting the CSV

In [ ]:
## Downloading the IBM HR Analytics dataset using the requests module (version 2.32.5)

# Importing the requests library
import requests
url = 'https://github.com/devtlv/Datasets-DA-Bootcamp-2-/raw/refs/heads/main/Week%208%20-%20Hackathon%20&%20AB%20Testing/W8D1%20-%20Hackathon/IBM%20HR%20Analytics%20Employee%20Attrition%20&%20Performance.zip'

response = requests.get(url)
file_Path = 'IBM_HR_Analytics_Employee_Attrition_Performance.zip'

if response.status_code == 200:
    with open(file_Path, 'wb') as file:
        file.write(response.content)
    print('File downloaded successfully')
else:
    print('Failed to download file')

File downloaded successfully


In [ ]:
## Extracting the IBM HR Analytics csv file from the archive and moving it to a "dataset" folder using
## the zipfile and os modules (dropping unnecessary folder cascading from the zip).

# Importing libraries
import zipfile
import os

zip_path = "IBM_HR_Analytics_Employee_Attrition_Performance.zip"
file_to_extract = "IBM HR Analytics Employee Attrition & Performance/IBM HR Analytics Employee Attrition & Performance/WA_Fn-UseC_-HR-Employee-Attrition.csv"   # the exact internal path
output_dir = "dataset"

# Making sure the output folder exists
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(file_to_extract) as source_file:
        # Get only the filename (drop folders)
        filename = os.path.basename(file_to_extract)
        output_path = os.path.join(output_dir, filename)

        # Write file contents
        with open(output_path, 'wb') as target_file:
            target_file.write(source_file.read())

print(f"Extracted {file_to_extract} → {output_path}")

Extracted IBM HR Analytics Employee Attrition & Performance/IBM HR Analytics Employee Attrition & Performance/WA_Fn-UseC_-HR-Employee-Attrition.csv → dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv


## b. Loading the csv dataset into a dataframe with pandas

In [2]:
## Loading the IBM HR Analytics dataset in a dataframe

# Importing the pandas library
import pandas as pd

# Creating the dataframe from the csv
df = pd.read_csv("dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")

## c. Initial high-level data exploration

In [3]:
# Checking first five entries of the dataset with .head()
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [10]:
# Checking structure and datatypes with .info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

> 👉 The dataset has 1470 entries and 35 columns

> 👉 Conclusions and recommendations regarding data transformation to perform:

| Columns currently int64                                                                                                                                                                                                | Current dtype | Suggested dtype                                | Reason                                                                                                              |
| -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------- | ---------------------------------------------- | ------------------------------------------------------------------------------------------------------------------- |
| `Age`                                                                                                                                                                                                    | int64         | ✅ keep as int64                                | Continuous numeric variable.                                                                                        |
| `DailyRate`, `HourlyRate`, `MonthlyIncome`, `MonthlyRate`                                                                                                                                                | int64         | ✅ keep as int64                                | Continuous numeric values.                                                                                          |
| `DistanceFromHome`, `PercentSalaryHike`, `YearsAtCompany`, `YearsInCurrentRole`, etc.                                                                                                                    | int64         | ✅ keep as int64                                | Continuous or count-based — fine as numeric.                                                                        |
| `Education`, `EnvironmentSatisfaction`, `JobInvolvement`, `JobLevel`, `JobSatisfaction`, `PerformanceRating`, `RelationshipSatisfaction`, `StockOptionLevel`, `TrainingTimesLastYear`, `WorkLifeBalance` | int64         | ⚠️ **convert to categorical (pd.Categorical)** | These are *ordinal* variables — coded as 1–4 or 1–5. Converting to categorical helps with readability and analysis. |
| `EmployeeCount`, `StandardHours`                                                                                                                                                                         | int64         | ⚠️ **drop or convert to constant**             | Usually constant across all rows (e.g., 1 or 80). They add no value — verify and drop if true.                      |
| `EmployeeNumber`                                                                                                                                                                                         | int64         | ⚠️ **treat as object/string (ID)**             | It’s an identifier, not numeric data — shouldn’t be used in math.                                                   |


| Columns currently object                                                                                             | Current dtype | Suggested dtype                       | Reason                                                                          |
| -------------------------------------------------------------------------------------------------- | ------------- | ------------------------------------- | ------------------------------------------------------------------------------- |
| `Attrition`                                                                                        | object        | ⚠️ **convert to boolean or category** | Typically “Yes” / “No”. Convert with `map({'Yes': 1, 'No': 0})` or to category. |
| `BusinessTravel`, `Department`, `EducationField`, `Gender`, `JobRole`, `MaritalStatus`, `OverTime` | object        | ⚠️ **convert to categorical**         | Nominal categorical variables — better memory and efficiency.                   |
| `Over18`                                                                                           | object        | ⚠️ **drop or set as constant**        | Usually all "Y". If so, drop (no information value).                            |


## d. Looking for missing values or duplicates

In [ ]:
# Looking for potential missing values in the dataframe (returning the sum of null values per column)
print(df.isnull().sum())
print(f"\n\nNumber of missing data in datateset: ", df.isnull().sum().sum())

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

> 👉 There are no missing values in the dataset

In [23]:
# Looking for potential duplicates in the dataframe (returning the count as an integer)
df.duplicated().sum()

np.int64(0)

> 👉 There are no duplicates in the dataset (returned value is 0)

## e. Converting and cleaning the dataset before analysis

# 2 ) Exploratory Data Analysis (EDA)

# 3 ) Correlation and Trend Analysis

# 4 ) Visualization

# 5 ) Summary of findings

# 6 ) Retention Strategy Development